In [ ]:
# hide
# dont_test
# default_exp handlers
%load_ext nb_black

<IPython.core.display.Javascript object>

# ASGIHandlers

In [ ]:
# export

from asgiref.sync import sync_to_async
from django.core.handlers.asgi import ASGIHandler


class AsyncFileASGIHandler(ASGIHandler):
    async def send_response(self, response, send):
        if response.is_async:
            return await response.stream(send)
        else:
            return super().send_response(response, send)

<IPython.core.display.Javascript object>

## Tests

In [ ]:
import django

from django.conf import settings

settings.configure()

RuntimeError: Settings already configured.

<IPython.core.display.Javascript object>

In [ ]:
class Response:
    is_async = True
    stream_called = False

    def __init__(self, content):
        self.content = content

    async def stream(self, send):
        self.stream_called = True
        return await send(self.content)


async def send(data):
    return data

<IPython.core.display.Javascript object>

In [ ]:
app = AsyncFileASGIHandler()
content = "<h1>Content!</h1>"
response = Response(content)
returned_content = await app.send_response(response, send)
assert response.stream_called
assert content == returned_content

<IPython.core.display.Javascript object>